In [2]:
from flowpipe import Graph, INode, Node, InputPlug, OutputPlug




In [3]:
class HireWorkers(INode):
    """A node can be derived from the INode interface.

    The plugs are defined in the init method.
    The compute method received the inputs from any connected upstream nodes.
    """

    def __init__(self, amount=None, **kwargs):
        super(HireWorkers, self).__init__(**kwargs)
        InputPlug('amount', self, amount)
        OutputPlug('workers', self)

    def compute(self, amount):
        workers = ['John', 'Jane', 'Mike', 'Michelle']
        print('{0} workers are hired to build the house.'.format(amount))
        return {'workers.{0}'.format(i): workers[i] for i in range(amount)}


@Node(outputs=['workers'])
def Build(workers, section):
    """A node can also be created by the Node decorator.outputs

    The inputs to the function are turned into InputsPlugs, otuputs are defined
    in the decorator itself. The wrapped function is used as the compute method.
    """
    print('{0} are building the {1}'.format(', '.join(workers.values()), section))
    return {'workers.{0}'.format(i): worker for i, worker in workers.items()}


@Node()
def Party(attendees):
    print('{0} and {1} are having a great party!'.format(
        ', '.join(list(attendees.values())[:-1]), list(attendees.values())[-1]))


# Create a graph with the necessary nodes
graph = Graph(name='How to build a house')
workers = HireWorkers(graph=graph, amount=4)
build_walls = Build(graph=graph, name='Build Walls', section='walls')
build_roof = Build(graph=graph, name='Build Roof', section='roof')
party = Party(graph=graph, name='Housewarming Party')

# Wire up the connections between the nodes
workers.outputs['workers']['0'].connect(build_walls.inputs['workers']['0'])
workers.outputs['workers']['1'].connect(build_walls.inputs['workers']['1'])
workers.outputs['workers']['2'].connect(build_roof.inputs['workers']['0'])
workers.outputs['workers']['3'].connect(build_roof.inputs['workers']['1'])
build_walls.outputs['workers']['0'] >> party.inputs['attendees']['0']
build_walls.outputs['workers']['1'] >> party.inputs['attendees']['2']
build_roof.outputs['workers']['0'] >> party.inputs['attendees']['1']
build_roof.outputs['workers']['1'] >> party.inputs['attendees']['3']
party.inputs['attendees']['4'].value = 'Homeowner'

/Users/pierthodo/anaconda3/envs/bounded_rationality/lib/python3.8/site-packages/flowpipe/node.py:67: UserWarning: Cannot serialize nodes defined in '__main__'
  warnings.warn("Cannot serialize nodes defined in '__main__'")


In [4]:
print(graph.name)
print(graph)
print(graph.list_repr())

How to build a house
+------------------------+          +------------------------+          +---------------------------+
|      HireWorkers       |          |       Build Roof       |          |    Housewarming Party     |
|------------------------|          |------------------------|          |---------------------------|
o amount<4>              |          o section<roof>          |          % attendees<>               |
|              workers<> %          % workers<>              |     +--->o  attendees.0<>            |
|           workers.0<>  o-----+--->o  workers.0<>           |     |--->o  attendees.1<>            |
|           workers.1<>  o-----|--->o  workers.1<>           |     |--->o  attendees.2<>            |
|           workers.2<>  o-----|    |              workers<> %     |--->o  attendees.3<>            |
|           workers.3<>  o-----|    |           workers.0<>  o-----|    o  attendees.4<Homeowner>   |
+------------------------+     |    |           workers.1<>  

In [5]:
graph.evaluate(mode='threading')  # Options are linear, threading and multiprocessing


4 workers are hired to build the house.
Mike, Michelle are building the roofJohn, Jane are building the walls

John, Jane, Mike, Michelle and Homeowner are having a great party!


In [6]:
graph = Graph(name='How to throw a birthday party')

@Node(outputs=['people'])
def InvitePeople(amount):
    people = ['John', 'Jane', 'Mike', 'Michelle']
    d = {'people.{0}'.format(i): people[i] for i in range(amount)}
    d['people'] = {people[i]: people[i] for i in range(amount)}
    return d

invite = InvitePeople(graph=graph, amount=4)
birthday_party = Party(graph=graph, name='Birthday Party')
invite.outputs['people'] >> birthday_party.inputs['attendees']

print(graph.name)
print(graph)
graph.evaluate(mode='threading')

How to throw a birthday party
+-------------------+          +---------------------+
|   InvitePeople    |          |   Birthday Party    |
|-------------------|          |---------------------|
o amount<4>         |     +--->o attendees<>         |
|          people<> o-----+    +---------------------+
+-------------------+                                 
John, Jane, Mike and Michelle are having a great party!


In [7]:
a = 5
b = 8
a >> b

0

In [8]:
a

5

In [9]:
b

8